# DINOv3 Deployment Tutorial

This tutorial guides you through exporting a DINOv3 model to ONNX format and running inference using ONNX Runtime. This is essential for deploying models to edge devices or production environments where Python availability might be restricted.

In [ ]:
import torch
import numpy as np
import onnxruntime as ort
from dinov3production import create_model
from dinov3production.deploy.exporter import EdgeExporter

model_name = 'dinov3_vits14'
print(f"Using model: {model_name}")

## 1. Export to ONNX
Create the model and use `EdgeExporter` to convert it.

In [ ]:
model = create_model(model_name, pretrained=True).eval()
exporter = EdgeExporter(model)

onnx_path = f'{model_name}.onnx'
exporter.to_onnx(onnx_path)
print(f"Model exported to {onnx_path}")

## 2. Inference with ONNX Runtime
Now we verify the exported model by running inference.

In [ ]:
# Create ONNX Runtime Session
session = ort.InferenceSession(onnx_path)

# Create Dummy Input
# ONNX expects numpy arrays
input_name = session.get_inputs()[0].name
input_shape = session.get_inputs()[0].shape
# Handle dynamic shapes if needed, usually [Batch, 3, H, W]
fake_input = np.random.randn(1, 3, 224, 224).astype(np.float32)

# Run Inference
outputs = session.run(None, {input_name: fake_input})

# Outputs is a list of results corresponding to output nodes
output_tensor = outputs[0]
print(f"Output Shape: {output_tensor.shape}")

# Compare with PyTorch output
with torch.no_grad():
    torch_out = model(torch.from_numpy(fake_input))
    
# Check Mean Squared Error
mse = np.mean((output_tensor - torch_out.numpy())**2)
print(f"MSE between ONNX and PyTorch: {mse:.6f}")